In [6]:
import numpy as np
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel
from dwave.system import LeapHybridCQMSampler
import pandas as pd
import urllib.request
from dotenv import load_dotenv, find_dotenv
import os

In [7]:
# Load Env Variables
load_dotenv(find_dotenv())
token = os.environ['DWAVE_API_KEY'] 

In [8]:
def parse_file(filename):
    column_names = ["Variable", "Player", "Position", "Rating"]
    players_df = pd.read_csv(filename, names=column_names)
    ratings = players_df['Rating'].tolist()
    return ratings, players_df

In [9]:
def build_cqm(): 
    
    # indicates that i player was selected
    p_labels = [f'{i}' for i in range(n_var)]
    cqm = ConstrainedQuadraticModel()


    # Set the objective: maximime the number of best rated players 
    objective = QuadraticModel()
    objective.add_linear_from(
                        (
                        (p_labels[i], -ratings[i]) for i in range(n_var) 
                        ), 
                        default_vartype='BINARY'
                        )

    cqm.set_objective(objective)

    # Select only 11 players
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(n_var) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=11, label='11 players', copy=False)

    # Select only 1 goalkeeper
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(2) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='1 goalkeeper', copy=False)

    # Select only 2 Central Defender
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(2,7) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=2, label='2 Central Defenders', copy=False)

    # Select only 1 Defender Left
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(7,8) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Defender Left', copy=False)

    # Select only 1 Defender Right
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(8,11) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Defender Right', copy=False)

    # Select only 1 Forward Striker
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(38,42) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Forward Striker', copy=False)

    # Select only 3 Midfielders
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(16,28) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=3, label='3 Midfielders', copy=False)

    # Select only 1 Forward Left
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(33,36) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Forward Left', copy=False)

    # Select only 1 Forward Right
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in range(36,38) 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Forward Right', copy=False)

    # Select Robertson in a position
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in robertson 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='Robertson in a position', copy=False)

    # Select Jota in a position
    constraint = QuadraticModel()
    constraint.add_linear_from(
                    (
                    (p_labels[i], 1) for i in jota 
                    ), 
                    default_vartype='BINARY'
                    )
    cqm.add_constraint(constraint, sense="==", rhs=1, label='Jota in a position', copy=False)
    return cqm

In [10]:
def sample (cqm):
    
    sampler = LeapHybridCQMSampler(token=token)
    print("Submitting CQM to solver {}.".format(sampler.solver.name))
    sampleset = sampler.sample_cqm(cqm, label='Example - Lineup')
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
    if not len(feasible_sampleset):
        raise ValueError("No feasible solution found")
    best = feasible_sampleset.first
    selected_item_indices = [key for key, val in best.sample.items() if val==1.0]
    selected_item_indices = [eval(i) for i in selected_item_indices]
    result =  players_df.filter(items = selected_item_indices, axis=0).drop('Variable', axis=1).sort_index().reset_index(drop=True)

    return result

In [11]:
# Read players rating from CSV
filename = 'https://raw.githubusercontent.com/dwave-examples/Line-up-optimization/master/players.csv'
ratings = parse_file(filename)[0]
players_df = parse_file(filename)[1]

In [12]:
n_var = players_df.shape[0]
print("\nBuilding a CQM for 11 amongst {} players.".format(str(n_var)))

robertson = [7,11,16]
jota = [27,32,34,37,41]

ratings_robertson = [e for i, e in enumerate(ratings) if i in robertson]
ratings_jota = [e for i, e in enumerate(ratings) if i in jota]


Building a CQM for 11 amongst 43 players.


In [13]:
cqm = build_cqm()

In [14]:
result = sample(cqm)
result

Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


,Player,Position,Rating
0,Alisson,GK,6.81
1,Phillips,DC,7.24
2,Fabinho,DC,7.11
3,Robertson,DL,6.85
4,Milner,DR,8.15
5,Williams,CM,7.77
6,Thiago,CM,7.38
7,Jota,CM,9.39
8,Mane,FWL,7.56
9,Salah,FWR,7.42


In [15]:
print(result['Rating'].sum())

82.67
